## Here we will work with UTKFace database

In [ ]:
import tensorflow as tf, pandas as pd, matplotlib.pyplot as plt, numpy as np, os
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input
from keras.utils import load_img, plot_model
from sklearn.model_selection import train_test_split

Get the images from UTKFace folder and store information about path, age and gender in lists:

In [ ]:
directory = os.getcwd() + '/UTKFace/'

# labels - age, gender, ethnicity
image_paths = []
age_labels = []
gender_labels = []

for filename in os.listdir(directory):
    image_path = os.path.join(directory, filename)
    temp = filename.split('_')
    age = int(temp[0])
    gender = int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)

In [ ]:
# convert to dataframe
df = pd.DataFrame()
df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels
df.head()

Display some exemplary images:

In [ ]:
# size of the whole grid of images
plt.figure(figsize=(10, 10))
files = df.iloc[0:25]

# display first 25 images in a 5x5 grid
for index, file, age, gender in files.itertuples():
    plt.subplot(5, 5, index + 1)
    img = load_img(file)
    img = np.array(img)
    plt.imshow(img)
    plt.title("Age: " + str(age) + " Gender: " + str(gender))
    plt.axis('off')
plt.show()

We extract the features and normalize the pixels in each of the images:

In [ ]:
x = []
for image in df['image']:
        img = load_img(image, color_mode = "grayscale")
        img = img.resize((128, 128), 3)
        img = np.array(img)
        x.append(img)
        
x = np.array(x)
x = x.reshape(len(x), 128, 128, 1)
x = x/255.0

In [ ]:
y_gender = np.array(df['gender'])
y_age = np.array(df['age'])

Build the model:

In [ ]:
# map this model so it can process both age and gender at the same time
# model = Sequential()

# model.add(Conv2D(32, kernel_size = (3, 3), input_shape = input_shape, activation = 'relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))

# model.add(Conv2D(64, kernel_size = (3, 3), input_shape = input_shape, activation = 'relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))

# model.add(Conv2D(128, kernel_size = (3, 3), input_shape = input_shape, activation = 'relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))

# model.add(Conv2D(256, kernel_size = (3, 3), input_shape = input_shape, activation = 'relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))

# model.add(Flatten())

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='relu'))

# model.compile(optimizer = 'adam', loss = 'mae', metrics = ['accuracy'])

In [ ]:
inputs = Input((128, 128, 1))

#instead of building model the classical way, we can add to separate hidden layers which would check for both age and gender and then pass to individual output layers
Conv2D_1 = Conv2D(32, kernel_size = (3, 3), strides = 1, padding = "same", activation = 'relu') (inputs)
MaxPooling2D_1 = MaxPooling2D(pool_size = (2, 2)) (Conv2D_1)
Conv2D_2 = Conv2D(64, kernel_size = (3, 3), strides = 1, padding = "same", activation = 'relu') (MaxPooling2D_1)
MaxPooling2D_2 = MaxPooling2D(pool_size = (2, 2)) (Conv2D_2)
Conv2D_3 = Conv2D(128, kernel_size = (3, 3), strides = 1, padding = "same", activation = 'relu') (MaxPooling2D_2)
MaxPooling2D_3 = MaxPooling2D(pool_size = (2, 2)) (Conv2D_3)
Conv2D_4 = Conv2D(256, kernel_size = (3, 3), strides = 1, padding = "same", activation = 'relu') (MaxPooling2D_3)
MaxPooling2D_4 = MaxPooling2D(pool_size = (2, 2)) (Conv2D_4)

Flatten = Flatten() (MaxPooling2D_4)

Dense_1 = Dense(256, activation='relu') (Flatten)
Dense_2 = Dense(256, activation='relu') (Flatten)

Dropout_1 = Dropout(0.3) (Dense_1)
Dropout_2 = Dropout(0.3) (Dense_2)

Dense_output_1 = Dense(1, activation='sigmoid', name='gender_out') (Dropout_1)
Dense_output_2 = Dense(1, activation='relu', name='age_out') (Dropout_2)

model = Model(inputs = [inputs], outputs = [Dense_output_1, Dense_output_2])

model.compile(loss=['binary_crossentropy', 'mae'], optimizer='adam', metrics=['accuracy'])

In [ ]:
# plot_model(model)

Training:

In [ ]:
history = model.fit(x = x, y = [y_gender, y_age], epochs = 20, validation_split = 0.2)

In [ ]:
acc = history.history['gender_out_accuracy']
val_acc = history.history['val_gender_out_accuracy']
epochs = range(len(acc))